In [193]:
import tensorflow as tf
import tensorflow_text as text
import numpy as np
import pandas as pd

import os

from IPython import display

ModuleNotFoundError: No module named 'tensorflow_text'

In [2]:
path = "rt-polaritydata/rt-polaritydata/"

pos_path = os.path.join(path, 'rt-polarity.pos')
neg_path = os.path.join(path, 'rt-polarity.neg')
print(pos_path, neg_path)

rt-polaritydata/rt-polaritydata/rt-polarity.pos rt-polaritydata/rt-polaritydata/rt-polarity.neg


In [3]:
def load_review(path, is_pos=True):
    with open(path) as f:
        review = pd.DataFrame({'review':f.read().splitlines()})
    review['sentiment'] = 1 if is_pos else 0
    return review

In [4]:
pos_review = load_review(pos_path, is_pos=True)
neg_review = load_review(neg_path, is_pos=False)

all_reviews = pd.concat([pos_review, neg_review])
all_reviews.head()

,review,sentiment
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


In [10]:
all_reviews.review_split = all_reviews.review.map(lambda line: line.split())

C:\Users\studia\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [43]:
from gensim.models import Word2Vec, KeyedVectors

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True)

In [49]:
model

In [56]:
queen = model['king'] - model['man'] + model['woman']
model.similar_by_word(queen)

[('king', 0.8449392318725586),
 ('queen', 0.7300517559051514),
 ('monarch', 0.6454660892486572),
 ('princess', 0.6156250834465027),
 ('crown_prince', 0.5818676352500916),
 ('prince', 0.577711820602417),
 ('kings', 0.5613664388656616),
 ('sultan', 0.5376776456832886),
 ('Queen_Consort', 0.5344247817993164),
 ('queens', 0.5289887189865112)]

### LSTM model

In [58]:
import operator
from functools import reduce

def create_vocabulary(df):
    splitted_reviews = df.review.map(lambda line: line.split())
    vocab = set(reduce(operator.add, splitted_reviews))
    return list(vocab)

vocab = create_vocabulary(all_reviews)

VOCAB_SIZE = len(vocab)
print('Total vocabluary size: {} words'.format(VOCAB_SIZE))

Total vocabluary size: 21425 words


In [96]:
word_to_number = {vocab[i]: i for i in range(VOCAB_SIZE)}
number_to_word = {i:vocab[i] for i in range(VOCAB_SIZE)}

In [97]:
df = all_reviews.copy()
df["review_splitted"] = df.review.map(lambda line: line.split())
df.head()

,review,sentiment,review_splitted
0,the rock is destined to be the 21st century's ...,1,"[the, rock, is, destined, to, be, the, 21st, c..."
1,"the gorgeously elaborate continuation of "" the...",1,"[the, gorgeously, elaborate, continuation, of,..."
2,effective but too-tepid biopic,1,"[effective, but, too-tepid, biopic]"
3,if you sometimes like to go to the movies to h...,1,"[if, you, sometimes, like, to, go, to, the, mo..."
4,"emerges as something rare , an issue movie tha...",1,"[emerges, as, something, rare, ,, an, issue, m..."


In [98]:
df["review_enc"] = df.review_splitted.map(lambda line: \
                                          [word_to_number[word] for word in line])

In [162]:
import tensorflow_datasets as tfds

dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset_tf, test_dataset = dataset['train'], dataset['test']

In [138]:
from sklearn.model_selection import train_test_split

X = df.review_enc.values.tolist()
y = df.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X_train), y_train.shape, len(X_test), y_test.shape

(8529, (8529,), 2133, (2133,))

In [188]:
train_dataset = tf.data.Dataset.from_tensor_slices((tf.ragged.constant(X_train), 
                                                    y_train.values))
test_dataset = tf.data.Dataset.from_tensor_slices((tf.ragged.constant(X_test), 
                                                   y_test.values))

In [189]:
BATCH_SIZE = 32

train_dataset = train_dataset.batch(BATCH_SIZE) # it should be padded_batch ...
test_dataset = test_dataset.batch(BATCH_SIZE)

In [191]:
for feature, target in train_dataset_tf.take(1):
    print('Feature: {}, target: {}'.format(feature, target))

Feature: [ 249    4  277  309  560    6 6639 4574    2   12   31 7759 3525 2128
   93 2306   43 2312 2527    6   30 1334 8044   24   10   16   10   17
  977   30  815 3339   41  841 7911  376 7974 1923    6  607  219   44
  242 1603   11 4329  102 2821 1139    2  969  161   51   18    4 5844
 2820  123 4493   40    6 4571   13  117   35  289  850  455   50  460
 6359 1069  343   20    1 3733 3511 7670    3  147    4  336    2   42
   18    4 3422  409 3533  871 2836  311    5 5080 1209    3  183  117
   35 1187    5 1955   11    1  226 7745    3  183 1466 7359 7961 1466
  665    2 6854 3178 1377 6266 1447  297    2 5797   36 4740  847 8050
    2    5 1929 1631 5986   22 5541 5688    5 1036 3746 8050    3   69
  264   35    4 4224    2   26   42   18    4  474 7968    8 1626   24
   10   16   10   17  134   15    9    1  435   13    9   55  598 2357
   48   30 6611 8044    3  500    1  101    6 2793    2 6548 7961 5861
  311    9  139 5398 2270   11 3674 1642   25 7796 7961 5012    3  4

In [192]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(None,), dtype='int64', ragged=True),
    tf.keras.layers.ToDense(pad_value=0, mask=True),
    tf.keras.layers.Embedding(VOCAB_SIZE, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

AttributeError: module 'tensorflow_core.keras.layers' has no attribute 'ToDense'

In [179]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [180]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, 
                    validation_steps=30)

ValueError: The two structures don't have the same nested structure.

First structure: type=RaggedTensorSpec str=RaggedTensorSpec(TensorShape([None, None, None]), tf.int32, 2, tf.int64)

Second structure: type=Tensor str=Tensor("embedding_1_input:0", shape=(None, None), dtype=float32)

More specifically: Substructure "type=RaggedTensorSpec str=RaggedTensorSpec(TensorShape([None, None, None]), tf.int32, 2, tf.int64)" is a sequence, while substructure "type=Tensor str=Tensor("embedding_1_input:0", shape=(None, None), dtype=float32)" is not
Entire first structure:
.
Entire second structure:
.

In [198]:
!pip install tensorflow-text # try on linux! and save trained model !!

ERROR: Could not find a version that satisfies the requirement tensorflow-text (from versions: none)
ERROR: No matching distribution found for tensorflow-text
